In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/bank-customer-churn-modeling/Churn_Modelling.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.drop('RowNumber',axis = 1 , inplace = True)
df.head()

In [ ]:
plt.figure(figsize = (10,8))
plt.bar(df['Exited'],df['Tenure'] )
plt.xlabel('Exited')
plt.ylabel('Tenure')
plt.title('Tenure vs Exited Graph')
plt.show()

In [ ]:
def unique_values_and_length(df):
    for column in df : 
        if df[column].dtypes == 'object':
            print(f'{column}:{df[column].unique()}:{len(df[column].unique())}')
unique_values_and_length(df)

In [ ]:
s = (df.dtypes == 'object')
object_cols = list(s[s].index)
print('Categorical Variables: ', object_cols)

In [ ]:
df1 = df.copy() 

from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()


df1[object_cols] = encoder.fit_transform(df1[object_cols])


df1.head()

In [ ]:
scaled_columns = ['CreditScore','Age','Balance','EstimatedSalary']

from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()

df1[scaled_columns] = scalar.fit_transform(df1[scaled_columns])

df1.head()


In [ ]:
features = df1.drop(['CustomerId','Exited'],axis = 1)
label = df1['Exited']

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(features,label,test_size = 0.2 ,random_state = 1)

print('x train shape: ',x_train.shape)
print('y train shape: ',y_train.shape)
print('x test shape: ',x_test.shape)
print('y test shape: ',y_test.shape)

In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier()

model.fit(x_train,y_train)
model.score(x_test,y_test)

In [ ]:
y_pred = model.predict(x_test)
y_pred[:5]

In [ ]:
y_test[:5]

In [ ]:
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix

print(classification_report(y_test,y_pred))

In [ ]:
score = accuracy_score(y_test,y_pred)
print('Accuracy Score : ', score)

Problem with this model is , 
In recall you will find 1's value as 0.24 which is bad simalarly the f1 score of 1 is bad . 


This is because , we have an imbalance dataset ,where in Exited we have '0': 7963 values,
and '1' values are: 2037 . 
So we can see model will create a biased opinion towards 0 .

To solve this problem we have three techniques : 
1) Undersample the 0 values to the shape of 1 ,
check the code below 

In [ ]:
df2 = df1.copy()

In [ ]:
df2['Exited'].value_counts()

In [ ]:
exited_0 = df2[df2['Exited'] == 0]
exited_1 = df2[df2['Exited'] == 1]

print('Shape of exited_0: ',exited_0.shape)
print('Shape of exited_1: ',exited_1.shape)

In [ ]:
exited_o_undersample = exited_0.sample(len(exited_1))
print("Exited_0 under sample: ",exited_o_undersample.shape)

In [ ]:
df2_exited = pd.concat([exited_1,exited_o_undersample],axis =0)
df2_exited['Exited'].value_counts()

In [ ]:
df2_exited.head()

In [ ]:
df2_feature = df2_exited.drop(['CustomerId','Exited'],axis = 1)
df2_label = df2_exited['Exited']

In [ ]:
df2_feature.shape

In [ ]:
df2_label.shape

In [ ]:
from sklearn.model_selection import train_test_split

df2_x_train,df2_x_test,df2_y_train,df2_y_test = train_test_split(df2_feature,df2_label,test_size = 0.2 ,random_state = 1,stratify = df2_label)

print('df2_x_train shape: ',df2_x_train.shape)
print('df2_y_train shape: ',df2_y_train.shape)
print('df2_x_test shape: ',df2_x_test.shape)
print('df2_y_test shape: ',df2_y_test.shape)

In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier()

model.fit(df2_x_train,df2_y_train)
model.score(df2_x_test,df2_y_test)

In [ ]:
df2_y_pred = model.predict(df2_x_test)

In [ ]:
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix

print(classification_report(df2_y_test,df2_y_pred))

Now by this method we got less accuracy , so we will try second method of oversample the 1 values to the shape of 0 

In [ ]:
df2['Exited'].value_counts()

In [ ]:
df3 = df2.copy()

In [ ]:
exited_0 = df3[df3['Exited'] == 0]
exited_1 = df3[df3['Exited'] == 1]

print('Shape of exited_0: ',exited_0.shape)
print('Shape of exited_1: ',exited_1.shape)

In [ ]:
df3_exited_1_over = exited_1.sample(len(exited_0),replace = True)
df3_exited_1_over.shape

In [ ]:
df3 = pd.concat([exited_0,df3_exited_1_over],axis =0)
df3['Exited'].value_counts()

In [ ]:
df3.head()

In [ ]:
df3_feature = df3.drop(['CustomerId','Exited'],axis = 1)
df3_label = df3['Exited']

In [ ]:
df3_feature.shape

In [ ]:
df3_label.shape

In [ ]:
from sklearn.model_selection import train_test_split

df3_x_train,df3_x_test,df3_y_train,df3_y_test = train_test_split(df3_feature,df3_label,test_size = 0.2 ,random_state = 1,stratify = df3_label)

print('df3_x_train shape: ',df3_x_train.shape)
print('df3_y_train shape: ',df3_y_train.shape)
print('df3_x_test shape: ',df3_x_test.shape)
print('df3_y_test shape: ',df3_y_test.shape)

In [ ]:
import tensorflow as tf 
from tensorflow import keras 

In [ ]:
model = tf.keras.Sequential([
    keras.layers.Dense(30,input_shape= (11,),activation = 'relu'),
    keras.layers.Dense(30,activation = 'relu'),
    keras.layers.Dense(1,activation = 'sigmoid')    
])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)


model.fit(df3_x_train,df3_y_train,epochs = 100)

In [ ]:
model.evaluate(df3_x_test,df3_y_test)

In [ ]:
df3_yp = model.predict(df3_x_test)

In [ ]:
df3_yp[:5]

In [ ]:
df3_y_pred = []

for element in df3_yp : 
    if element > 0.5 : 
        df3_y_pred.append(1)
    else : 
        df3_y_pred.append(0)

In [ ]:
df3_y_pred[:5]

In [ ]:
df3_y_test[:5]

In [ ]:
from sklearn.metrics import classification_report,accuracy_score

print(classification_report(df3_y_test,df3_y_pred))

In [ ]:
score = accuracy_score(df3_y_test,df3_y_pred)
print('Accuracy Score: ',score)

Third method is SMOTE 

In [ ]:
df4_feature = features.copy()
df4_label = label.copy()

In [ ]:
df4_feature.sample(5)

In [ ]:
df4_label.sample(5)

In [ ]:
df4_label.value_counts()

In [ ]:
from imblearn.over_sampling import SMOTE    

In [ ]:
smote = SMOTE(sampling_strategy = 'minority')
feature_smote , label_smote = smote.fit_resample(df4_feature,df4_label)

label_smote.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

df4_x_train,df4_x_test,df4_y_train,df4_y_test = train_test_split(feature_smote,label_smote,test_size = 0.2 ,random_state = 1,stratify = df3_label)

print('df3_x_train shape: ',df3_x_train.shape)
print('df3_y_train shape: ',df3_y_train.shape)
print('df3_x_test shape: ',df3_x_test.shape)
print('df3_y_test shape: ',df3_y_test.shape)

In [ ]:
import tensorflow as tf 
from tensorflow import keras 

In [ ]:
model = tf.keras.Sequential([
    keras.layers.Dense(30,input_shape= (11,),activation = 'relu'),
    keras.layers.Dense(30,activation = 'relu'),
    keras.layers.Dense(1,activation = 'sigmoid')    
])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)


model.fit(df4_x_train,df4_y_train,epochs = 200)

In [ ]:
model.evaluate(df4_x_test,df4_y_test)

In [ ]:
df4_yp = model.predict(df4_x_test)
df4_yp[:10]

In [ ]:
df4_y_pred = []

for element in df4_yp: 
    if element > 0.5 : 
        df4_y_pred.append(1)
    else : 
        df4_y_pred.append(0)


In [ ]:
print(df4_y_test[:5])

In [ ]:
print(df4_y_pred[:5])

In [ ]:
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix

print(classification_report(df4_y_test,df4_y_pred))

In [ ]:
cm = confusion_matrix(df4_y_test,df4_y_pred)
plt.figure(figsize = (5,4))
sns.heatmap(cm,annot = True)

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')

plt.show()

In [ ]:
score = accuracy_score(df4_y_test,df4_y_pred)
print('Acc Score: ' , score)